In [1]:
!pip install pandavro
import pandas as pd
import pandavro as pdx
import json

df_history = pdx.read_avro("../data/history.avro")
with open("../data/dictionary.json") as fp:
    dictionary = json.load(fp)
df_timestamp = pd.read_json("../data/metadata.json", lines=True)

keys = [elem for elem in dictionary]
values = [dictionary[idx] for idx in keys]
df_dict = pd.DataFrame({"document" : keys, "pos" : values})

df_history = df_history.merge(df_dict, on="document")

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
     |████████████████████████████████| 430kB 9.4MB/s eta 0:00:01


In [2]:
df_history["date"] = pd.to_datetime(df_history["timestamp"], format="%Y-%m-%dT%H:%M:%S")
df_history["date"].max()

Timestamp('2019-02-28 23:00:00')

In [3]:
df_history[df_history["pos"] == 526570]

,document,id,timestamp,pos,date
11878679,@eastgo_742,#5afa8aaa942630fde1b6a0e087199f9d,2018-11-16T18:00:00,526570,2018-11-16 18:00:00
11878680,@eastgo_742,#112de22d913b8c10edef90c5b2f4cadb,2019-01-08T10:00:00,526570,2019-01-08 10:00:00
11878681,@eastgo_742,#5bdf8768f25494fadc387f8a52d6150f,2018-11-29T03:00:00,526570,2018-11-29 03:00:00
11878682,@eastgo_742,#612d09c6596919f63d57e962d825a932,2018-11-19T01:00:00,526570,2018-11-19 01:00:00
11878683,@eastgo_742,#0fb9b9ff48e3951ba0019962f8103fa2,2019-02-15T12:00:00,526570,2019-02-15 12:00:00
11878684,@eastgo_742,#d9635d8b2f91f3631500e4d99aa63988,2018-12-13T23:00:00,526570,2018-12-13 23:00:00
11878685,@eastgo_742,#2290167a4fe7bb0f1d4c62a86c708111,2019-01-18T20:00:00,526570,2019-01-18 20:00:00
11878686,@eastgo_742,#5e0d40b3945725e67a05adc17a7372ee,2018-12-05T23:00:00,526570,2018-12-05 23:00:00
11878687,@eastgo_742,#677e984e245b344f61dc5d3cc1f352c8,2019-02-25T23:00:00,526570,2019-02-25 23:00:00
11878688,@eastgo_742,#072b7bdd2bbd58334528ab7061350d23,2019-01-30T19:00:00,526570,2019-01-30 19:00:00


In [2]:
df = df_history.groupby("id")

In [3]:
df_history

,document,id,timestamp,pos
0,@moondol_209,#cfbda49fbe26a2668b00428ca561d9a7,2019-02-18T00:00:00,195872
1,@moondol_209,#d2882e2d92af7b9603df8d3496478a6a,2019-01-12T17:00:00,195872
2,@moondol_209,#50001d770ccdc6c4ee2739328ae60051,2019-01-12T13:00:00,195872
3,@moondol_209,#a30676fa0756df1ee7ebe956db493d71,2019-02-06T15:00:00,195872
4,@moondol_209,#20c9d2f2f557f943bdaaf41d45ed644b,2019-01-27T20:00:00,195872
5,@moondol_209,#7785ece1084786c70b00942b0c9a0cb6,2019-01-12T19:00:00,195872
6,@moondol_209,#8b6b67701c2ad8f8e604a0c9cd1b5209,2019-01-12T22:00:00,195872
7,@moondol_209,#e8bc8a922e7596221a9289f31ce80db9,2019-02-21T21:00:00,195872
8,@moondol_209,#a1684b1d8d625fe3ccaf463d96164465,2019-02-25T02:00:00,195872
9,@moondol_209,#5548c6d5aca91bdca91c348238429956,2019-01-12T15:00:00,195872


In [4]:
df_history["date"] = pd.to_datetime(df_history["timestamp"], format="%Y-%m-%dT%H:%M:%S")

In [5]:
import numpy as np

def pad_array(x, flag):
    if len(x) == 15:
        return x
    else:        
        to_pad = [x[0]] * (15 - len(x))
        return to_pad + x
        

def top_n(row):
    name = row[0]
    frame = row[1]

    df_train = frame[(frame['date'] < '2019-02-22')]
    df_train = df_train.sort_values(by=["timestamp", "document"])
    train_array = list(df_train["pos"].values)
    train_length = len(train_array)
    
    df_eval = frame[(frame['date'] >= '2019-02-22')]
    df_eval = df_eval.sort_values(by=["timestamp", "document"])
    eval_array = list(df_eval["pos"].values)
    eval_length = len(eval_array)
    
    if train_length == 0:
        train_array = list(eval_array)
        return name, np.array(pad_array(train_array[-15:], 1)), np.array(eval_array)
    
    if eval_length == 0:
        if train_length >= 20:
            train_array = train_array[-20:-5]
            eval_array = train_array[-5:]
        elif train_length < 20 and train_length > 15:
            x = train_length - 15
            train_array = train_array[-train_length:-x]
            eval_array = train_array[-x:]
        else:
            if train_length != 1:
                train_array = train_array[-train_length:-1]
            eval_array = train_array[-1:]
        return name, np.array(pad_array(train_array[-15:], 2)), np.array(eval_array)
    
    return name, np.array(pad_array(train_array[-15:], 3)), np.array(eval_array)
        

In [6]:
import multiprocessing
from tqdm import tqdm

groupped = df_history.groupby("id")
cores = multiprocessing.cpu_count()

with multiprocessing.Pool(cores) as p:
    res = list(tqdm(p.imap(top_n, groupped), total=len(groupped)))

100%|██████████| 302391/302391 [02:59<00:00, 1686.94it/s] 


In [12]:
length_list = [len(elem[2])==15 for elem in res]

In [16]:
stripped = [(elem[0], elem[1], elem[2][0:5]) for elem in res]

In [27]:
df = pd.DataFrame(res, columns=["id", "train", "eval"])

In [28]:
df.to_parquet("../data/test.parquet")

In [29]:
len(res)

302391

In [30]:
df = pd.DataFrame(stripped, columns=["id", "train", "eval"])

In [31]:
df.to_parquet("../data/train.parquet")

In [19]:
temp = pd.DataFrame({
    "id" : [[1, 2, 3]]
})

In [38]:
temp = dd.from_pandas(temp, npartitions=1)


In [40]:
temp.compute()

,id
0,"[1, 2, 3]"


In [28]:
df_train = dask_frame[(dask_frame['date'] < '2019-02-22')]
df_eval = dask_frame[(dask_frame['date'] >= '2019-02-22')]

In [31]:
type(df_train.compute())

pandas.core.frame.DataFrame

In [80]:
array = [1]
array[-1:-1]

[]

In [49]:
import numpy as np

df = pd.DataFrame({'x': [1, 2, 3, 4, 5],
                   'y': [1., 2., 3., 4., 5.]})
ddf = dd.from_pandas(df, npartitions=2)

def myadd(row, a, b=1):
    return np.array(row), np.array(row) + a

res = ddf.apply(myadd, axis=1, args=(2,), b=1.5) 

/home/jupyter/.local/lib/python3.5/site-packages/dask/dataframe/core.py:3966: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'object'))

  warnings.warn(meta_warning(meta))


In [63]:
res = res.compute()

In [67]:
res[0][1]

array([3., 3.])

In [59]:
def pad_array(x):
    if len(x) == 15:
        return x
    else:
        to_pad = [x[0]] * (15 - len(x))
        return to_pad + x
    
len(pad_array(array))

15

In [ ]:
temp = pd.D